In [146]:
import pandas as pd
import string
from nltk import pos_tag
from nltk.stem import WordNetLemmatizer
from nltk import word_tokenize
from nltk.corpus import wordnet
import nltk

import spacy
nlp = spacy.load('en')

import re

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords
import numpy as np
from scipy.sparse.csr import csr_matrix

from sklearn.feature_extraction.text import TfidfTransformer

In [2]:
contr = pd.read_csv("./materials/contractions.csv")
contr.head()

,short,long
0,ain t,are not
1,aren t,are not
2,can t,cannot
3,can t ve,cannot have
4,'cause,because


In [3]:
contractions = {}
for key, val in zip(contr['short'],contr['long']):
    contractions[key] = val

In [383]:
basic_vocabulary = []
with open("./materials/A1_vocab_processed.txt", "r") as voc:
    for word in voc.readlines():
        basic_vocabulary.append(word[:-1])
#basic_vocabulary = set(basic_vocabulary)
#basic_vocabulary

adjectives = []
with open("./materials/common_adj.txt", "r") as common_adj:
    for word in common_adj.readlines():
        adjectives.append(word[:-1])
        
common_uncountable = []
with open("./materials/common_unountable_manually_filtered.txt", "r") as common_unctbl:
    for word in common_unctbl.readlines():
        common_uncountable.append(word[:-1])

countries = []
with open("./materials/countries.txt", "r") as cntr:
    for word in cntr.readlines():
        countries.append(word[:-1])

names = []
with open("./materials/names.txt", "r") as names_file:
    for word in names_file.readlines():
        names.append(word[:-1])
        
print(len(basic_vocabulary), len(adjectives), len(common_uncountable), len(countries), len(countries), len(names))
final_basic_vocabulary = basic_vocabulary
final_basic_vocabulary.extend(adjectives)
final_basic_vocabulary.extend(common_uncountable)
final_basic_vocabulary.extend(countries)
final_basic_vocabulary.extend(names)
len(final_basic_vocabulary)

660 50 81 196 196 3309


4296

In [46]:
texts = pd.read_excel("./materials/texts.xlsx")
texts.head()

,id,Title,Text
0,1,Jimmy’s Breakfast,This is my friend Jimmy. He is from India. Jim...
1,2,A Girl from Green Valley,Green Valley was a small village near the Icy ...
2,3,Alice in Wonderland,Alice was beginning to get very tired of sitti...
3,4,Volkswagen,Volkswagen's emissions cheating will cost the ...
4,5,My Friends,My name is Sonya. I am 20 and I come from Moro...


In [17]:
puncuation_primary = string.punctuation + "’" +"“" + "”" 
puncuation_primary = puncuation_primary.replace(".","")

In [107]:
sent_detector = nltk.data.load('tokenizers/punkt/english.pickle')
test_line = "I can't  visit U.S. and I couldn't   to go by your daddy's car. I want to say hello to her brother."#’'
lemmatizer = WordNetLemmatizer()
def punct_setnence_lower (line, sent_detector, debug = False, deep_debug = False):
    initial_sentences_list = sent_detector.tokenize(line.strip())
    final_sentences_list = []
    
    for sentence in initial_sentences_list:
        if(debug): print("sentence:", sentence)
        cleaned_line = ''
        #clean by word
        for word in sentence.split():
            if(debug): print(word)
            clean_word =''
            for char in word:
                if(deep_debug): print("char before cleaning:", char)
                if char not in puncuation_primary:
                    clean_word += char.lower()
                    if(deep_debug): print("char after cleaning:", char.lower())
                else:
                    clean_word += ' '
                    if(deep_debug): print("char has been deleted")
            clean_word = clean_word.lstrip()
            if(debug):print("non punctuation and lower:",clean_word)       

            if (clean_word in contractions):
                if(debug):print("word before contractions parsing:",clean_word)
                clean_word = ''.join(contractions[clean_word])
                if(debug):print("word after contractions parsing:",clean_word)
            if(debug): print("finally cleaned word/s:",clean_word)    
            cleaned_line += clean_word + ' '
        #handle abbreviations delete dots only in the end    
        for char_ind in range(len(cleaned_line) - 3, len(cleaned_line)):
            max_dot_index = len(cleaned_line)
            if cleaned_line[char_ind] == ".":
                if(char_ind < max_dot_index): max_dot_index = char_ind
                break
        cleaned_line = cleaned_line[:max_dot_index]    
        cleaned_line = re.sub(' +', ' ', cleaned_line)
        final_sentences_list.append(cleaned_line)
    return final_sentences_list
            
clean_line = punct_setnence_lower(test_line, sent_detector,debug = False)
clean_line

['i cannot visit u.s. and i could not to go by your daddy s car',
 'i want to say hello to her brother']

In [108]:
processed_ex = punct_setnence_lower(texts["Text"][0], sent_detector)

In [109]:
processed_ex

['this is my friend jimmy',
 'he is from india',
 'jimmy is a vegetarian',
 'the best breakfast for him is a glass of orange juice two apples and three bananas',
 'jimmy likes watermelons very much',
 'a watermelon is green outside and red inside',
 'it is hard and soft inside',
 'it is juicy and sweet',
 'what a lovely fruit ',
 'a month ago sally received a letter',
 'it was an invitation for a wedding',
 'her sister victoria is getting married',
 'last year sally saw such a ceremony',
 'it took place on saturday',
 'there were a lot of guests people wore nice clothes met a husband and a wife near the church threw confetti ate a very tasty cake made videos and took photos',
 'in her letter victoria asked sally to be her bridesmaid',
 'it means that on the day of wedding sally must help victoria when she puts her wedding dress on and brushes her hair',
 'sally must also accompany victoria on her way to the church and carry a traditional bunch of flowers',
 'sally agreed with pleasure 

In [61]:
doc = nlp(processed_ex[0])
for token in doc:
    print(token.text, token.lemma_, token.pos_, token.tag_, token.dep_,
          #token.shape_, token.is_alpha, token.is_stop,
          nlp.vocab.morphology.tag_map[token.tag_])

this this DET DT nsubj {74: 89}
is be VERB VBZ ROOT {74: 99, 'VerbForm': 'fin', 'Tense': 'pres', 'Number': 'sing', 'Person': 3}
my -PRON- ADJ PRP$ poss {74: 83, 'PronType': 'prs', 'Poss': 'yes'}
friend friend NOUN NN attr {74: 91, 'Number': 'sing'}
jimmy jimmy NOUN NN dep {74: 91, 'Number': 'sing'}


In [74]:
doc = nlp("this is my friends jimmy and catt they are swimmers")
for token in doc:
    if(token.lemma_[0] == "-"):
        token.lemma_ = token.text
    print(token.text, token.lemma_, nlp.vocab.morphology.tag_map[token.tag_])

this this {74: 89}
is be {74: 99, 'VerbForm': 'fin', 'Tense': 'pres', 'Number': 'sing', 'Person': 3}
my my {74: 83, 'PronType': 'prs', 'Poss': 'yes'}
friends friend {74: 91, 'Number': 'plur'}
jimmy jimmy {74: 91, 'Number': 'sing'}
and and {74: 88, 'ConjType': 'coor'}
catt catt {74: 91, 'Number': 'sing'}
they they {74: 94, 'PronType': 'prs'}
are be {74: 99, 'VerbForm': 'fin', 'Tense': 'pres'}
swimmers swimmer {74: 91, 'Number': 'plur'}


In [82]:
sent = "this is my friends jimmy and catt they are swimmers"
spl = sent.split(" ")
len(doc) == len(spl)


True

In [111]:
def get_sent_gramm_features_map(sentence):
    split_sent_list = sentence.split()
    grammar_map = [None] * len(split_sent_list)  
    parsed_sentence =  nlp(sentence)   
    assert (len(parsed_sentence) == len(grammar_map))
    for gramm_ind in range(len(split_sent_list)):
        if(parsed_sentence[gramm_ind].lemma_[0] == "-"):
            parsed_sentence[gramm_ind].lemma_ = parsed_sentence[gramm_ind].text
        grammar_map[gramm_ind] = (parsed_sentence[gramm_ind].lemma_, nlp.vocab.morphology.tag_map[parsed_sentence[gramm_ind].tag_])
        
    return grammar_map

get_sent_gramm_features_map(sent)

[('this', {74: 89}),
 ('be',
  {74: 99, 'Number': 'sing', 'Person': 3, 'Tense': 'pres', 'VerbForm': 'fin'}),
 ('my', {74: 83, 'Poss': 'yes', 'PronType': 'prs'}),
 ('friend', {74: 91, 'Number': 'plur'}),
 ('jimmy', {74: 91, 'Number': 'sing'}),
 ('and', {74: 88, 'ConjType': 'coor'}),
 ('catt', {74: 91, 'Number': 'sing'}),
 ('they', {74: 94, 'PronType': 'prs'}),
 ('be', {74: 99, 'Tense': 'pres', 'VerbForm': 'fin'}),
 ('swimmer', {74: 91, 'Number': 'plur'})]

In [387]:
grammar_map = []
for sentence in processed_ex:
    sent_grammar_map = get_sent_gramm_features_map(sentence)
    grammar_map.append(sent_grammar_map)
grammar_map

[[('this', {74: 89}),
  ('be',
   {74: 99,
    'Number': 'sing',
    'Person': 3,
    'Tense': 'pres',
    'VerbForm': 'fin'}),
  ('my', {74: 83, 'Poss': 'yes', 'PronType': 'prs'}),
  ('friend', {74: 91, 'Number': 'sing'}),
  ('jimmy', {74: 91, 'Number': 'sing'})],
 [('he', {74: 94, 'PronType': 'prs'}),
  ('be',
   {74: 99,
    'Number': 'sing',
    'Person': 3,
    'Tense': 'pres',
    'VerbForm': 'fin'}),
  ('from', {74: 84}),
  ('india', {74: 91, 'Number': 'sing'})],
 [('jimmy', {74: 91, 'Number': 'sing'}),
  ('be',
   {74: 99,
    'Number': 'sing',
    'Person': 3,
    'Tense': 'pres',
    'VerbForm': 'fin'}),
  ('a', {74: 89}),
  ('vegetarian', {74: 91, 'Number': 'sing'})],
 [('the', {74: 89}),
  ('good', {74: 83, 'Degree': 'sup'}),
  ('breakfast', {74: 91, 'Number': 'sing'}),
  ('for', {74: 84}),
  ('him', {74: 94, 'PronType': 'prs'}),
  ('be',
   {74: 99,
    'Number': 'sing',
    'Person': 3,
    'Tense': 'pres',
    'VerbForm': 'fin'}),
  ('a', {74: 89}),
  ('glass', {74: 91, 

In [116]:
grammar_map[0]

[('this', {74: 89}),
 ('be',
  {74: 99, 'Number': 'sing', 'Person': 3, 'Tense': 'pres', 'VerbForm': 'fin'}),
 ('my', {74: 83, 'Poss': 'yes', 'PronType': 'prs'}),
 ('friend', {74: 91, 'Number': 'sing'}),
 ('jimmy', {74: 91, 'Number': 'sing'})]

In [133]:
def get_lemm_text(text_gramm_map):
    lemm_text = []
    for sentence in text_gramm_map:
        print(sentence)
        sentence_lemm = ''
        for word in sentence:
            sentence_lemm += word[0] + ' '
        sentence_lemm = sentence_lemm[:-1]
        lemm_text.append(sentence_lemm)
        
    return lemm_text

ex_text_lemm = get_lemm_text(grammar_map)

[('this', {74: 89}), ('be', {74: 99, 'VerbForm': 'fin', 'Tense': 'pres', 'Number': 'sing', 'Person': 3}), ('my', {74: 83, 'PronType': 'prs', 'Poss': 'yes'}), ('friend', {74: 91, 'Number': 'sing'}), ('jimmy', {74: 91, 'Number': 'sing'})]
[('he', {74: 94, 'PronType': 'prs'}), ('be', {74: 99, 'VerbForm': 'fin', 'Tense': 'pres', 'Number': 'sing', 'Person': 3}), ('from', {74: 84}), ('india', {74: 91, 'Number': 'sing'})]
[('jimmy', {74: 91, 'Number': 'sing'}), ('be', {74: 99, 'VerbForm': 'fin', 'Tense': 'pres', 'Number': 'sing', 'Person': 3}), ('a', {74: 89}), ('vegetarian', {74: 91, 'Number': 'sing'})]
[('the', {74: 89}), ('good', {74: 83, 'Degree': 'sup'}), ('breakfast', {74: 91, 'Number': 'sing'}), ('for', {74: 84}), ('him', {74: 94, 'PronType': 'prs'}), ('be', {74: 99, 'VerbForm': 'fin', 'Tense': 'pres', 'Number': 'sing', 'Person': 3}), ('a', {74: 89}), ('glass', {74: 91, 'Number': 'sing'}), ('of', {74: 84}), ('orange', {74: 83, 'Degree': 'pos'}), ('juice', {74: 91, 'Number': 'sing'}), (

[('but', {74: 88, 'ConjType': 'coor'}), ('at', {74: 84}), ('home', {74: 91, 'Number': 'sing'}), ('she', {74: 94, 'PronType': 'prs'}), ('understand', {74: 99, 'VerbForm': 'fin', 'Tense': 'past'}), ('that', {74: 84}), ('she', {74: 94, 'PronType': 'prs'}), ('could', {74: 99, 'VerbType': 'mod'}), ('not', {74: 85, 'Degree': 'pos'}), ('fit', {74: 99, 'VerbForm': 'inf'}), ('into', {74: 84}), ('it', {74: 94, 'PronType': 'prs'})]
[('what', {74: 91, 'PronType': 'int|rel'}), ('a', {74: 89}), ('pity', {74: 91, 'Number': 'sing'})]
[('that', {74: 89}), ('be', {74: 99, 'VerbForm': 'fin', 'Tense': 'pres', 'Number': 'sing', 'Person': 3}), ('why', {74: 85, 'PronType': 'int|rel'}), ('sally', {74: 85, 'Degree': 'pos'}), ('be', {74: 99, 'VerbForm': 'fin', 'Tense': 'pres', 'Number': 'sing', 'Person': 3}), ('on', {74: 84}), ('a', {74: 89}), ('diet', {74: 91, 'Number': 'sing'}), ('now', {74: 85, 'Degree': 'pos'})]
[('she', {74: 94, 'PronType': 'prs'}), ('do', {74: 99, 'VerbForm': 'fin', 'Tense': 'pres', 'Numb

In [134]:
ex_text_lemm

['this be my friend jimmy',
 'he be from india',
 'jimmy be a vegetarian',
 'the good breakfast for him be a glass of orange juice two apple and three banana',
 'jimmy like watermelon very much',
 'a watermelon be green outside and red inside',
 'it be hard and soft inside',
 'it be juicy and sweet',
 'what a lovely fruit',
 'a month ago sally receive a letter',
 'it be an invitation for a wedding',
 'her sister victoria be get marry',
 'last year sally see such a ceremony',
 'it take place on saturday',
 'there be a lot of guest people wear nice clothe meet a husband and a wife near the church throw confetti eat a very tasty cake make video and take photo',
 'in her letter victoria ask sally to be her bridesmaid',
 'it mean that on the day of wedding sally must help victoria when she put her wedding dress on and brush her hair',
 'sally must also accompany victoria on her way to the church and carry a traditional bunch of flower',
 'sally agree with pleasure and buy a beautiful dress 

In [182]:
def sort_coo(coo_matrix):
    tuples = zip(coo_matrix.col, coo_matrix.data)
    return sorted(tuples, key=lambda x: (x[1], x[0]), reverse=True)

def extract_tfidf(feature_names, sorted_items):

    score_vals = []
    feature_vals = []
    
    # word index and corresponding tf-idf score
    for idx, score in sorted_items:
        
        #keep track of feature name and its corresponding score
        score_vals.append(round(score, 3))
        feature_vals.append(feature_names[idx])
 
    #create a tuples of feature,score
    #results = zip(feature_vals,score_vals)
    results= {}
    for idx in range(len(feature_vals)):
        results[feature_vals[idx]]=score_vals[idx]
    
    return results

In [184]:
def get_tf_idf_dict(lemm_lines_list):
    cv=CountVectorizer(max_df=0.85,stop_words=set(stopwords.words('english')))
    word_count_vector=cv.fit_transform(lemm_lines_list)
    
    tfidf_transformer=TfidfTransformer(smooth_idf=True,use_idf=True)
    tfidf_transformer.fit(word_count_vector)
    tf_idf_vector=tfidf_transformer.transform(cv.transform(ex_text_lemm))
    
    feature_names=cv.get_feature_names()
    
    sorted_items=sort_coo(tf_idf_vector.tocoo())
    
    words_tf_idf=extract_tfidf(feature_names,sorted_items)
    
    return words_tf_idf

In [324]:
tf_idf_dict = get_tf_idf_dict(ex_text_lemm)
tf_idf_dict

{'accompany': 0.323,
 'add': 0.39,
 'ago': 0.488,
 'agree': 0.371,
 'also': 0.323,
 'apple': 0.341,
 'ask': 0.514,
 'banana': 0.341,
 'beautiful': 0.415,
 'boil': 0.447,
 'breakfast': 0.341,
 'bridesmaid': 0.514,
 'brush': 0.298,
 'bunch': 0.323,
 'buy': 0.415,
 'cake': 0.189,
 'calorie': 0.39,
 'carry': 0.323,
 'ceremony': 0.371,
 'church': 0.207,
 'clothe': 0.232,
 'confetti': 0.232,
 'cook': 0.447,
 'could': 0.5,
 'day': 0.298,
 'diet': 0.844,
 'dieting': 0.707,
 'dream': 0.441,
 'dress': 0.266,
 'easy': 0.707,
 'eat': 0.176,
 'even': 0.573,
 'every': 0.408,
 'everyone': 0.457,
 'everything': 0.39,
 'fish': 0.447,
 'fit': 0.5,
 'flower': 0.323,
 'friend': 0.774,
 'fruit': 0.512,
 'fry': 0.505,
 'get': 0.529,
 'give': 0.441,
 'glass': 0.341,
 'go': 0.408,
 'good': 0.341,
 'great': 0.457,
 'green': 0.378,
 'guest': 0.232,
 'gym': 0.408,
 'hair': 0.298,
 'hard': 0.598,
 'help': 0.298,
 'home': 0.5,
 'husband': 0.232,
 'india': 1.0,
 'inside': 0.426,
 'invitation': 0.774,
 'jimmy': 0.43

In [358]:
phrasal_list = []
with open("phrasal_verbs.txt", "r") as pv_doc:
    for pv in pv_doc:
        phrasal_list.append(pv[:-1])
#phrasal_list

In [ ]:
все это делать внутри грамматической карты предложения

In [350]:
def get_weights_empty_list(gramm_map_text):
    weights_list = []
    for sentence in gramm_map_text:
        #print(sentence)
        sentence_weights = []
        for element in sentence:
            weight = {"word" : element[0], "weight": 0}
            sentence_weights.append(weight)
        weights_list.append(sentence_weights)
    return weights_list
test_weights = get_weights_empty_list(grammar_map_test)
test_weights

[[{'weight': 0, 'word': 'i'},
  {'weight': 0, 'word': 'have'},
  {'weight': 0, 'word': 'get'},
  {'weight': 0, 'word': 'marry'}],
 [{'weight': 0, 'word': 'i'},
  {'weight': 0, 'word': 'have'},
  {'weight': 0, 'word': 'go'},
  {'weight': 0, 'word': 'there'}],
 [{'weight': 0, 'word': 'i'},
  {'weight': 0, 'word': 'will'},
  {'weight': 0, 'word': 'be'},
  {'weight': 0, 'word': 'do'},
  {'weight': 0, 'word': 'that'}],
 [{'weight': 0, 'word': 'i'},
  {'weight': 0, 'word': 'make'},
  {'weight': 0, 'word': 'it'},
  {'weight': 0, 'word': 'up'}],
 [{'weight': 0, 'word': 'do'},
  {'weight': 0, 'word': 'that'},
  {'weight': 0, 'word': 'be'},
  {'weight': 0, 'word': 'not'},
  {'weight': 0, 'word': 'my'},
  {'weight': 0, 'word': 'hobby'}],
 [{'weight': 0, 'word': 'I'},
  {'weight': 0, 'word': 'be'},
  {'weight': 0, 'word': 'do'},
  {'weight': 0, 'word': 'it'},
  {'weight': 0, 'word': 'whole'},
  {'weight': 0, 'word': 'day'}],
 [{'weight': 0, 'word': 'I'},
  {'weight': 0, 'word': 'will'},
  {'weight

In [272]:
grammar_map_test

[[('i', {74: 94, 'PronType': 'prs'}),
  ('have', {74: 99, 'Tense': 'pres', 'VerbForm': 'fin'}),
  ('go', {74: 99, 'Aspect': 'perf', 'Tense': 'past', 'VerbForm': 'part'}),
  ('there', {74: 85, 'Degree': 'pos'})],
 [('i', {74: 94, 'PronType': 'prs'}),
  ('will', {74: 99, 'VerbType': 'mod'}),
  ('be', {74: 99, 'VerbForm': 'inf'}),
  ('do', {74: 99, 'Aspect': 'prog', 'Tense': 'pres', 'VerbForm': 'part'}),
  ('that', {74: 89})],
 [('i', {74: 94, 'PronType': 'prs'}),
  ('make', {74: 99, 'Tense': 'past', 'VerbForm': 'fin'}),
  ('it', {74: 94, 'PronType': 'prs'}),
  ('up', {74: 93})]]

In [273]:
test_weights

[[{'i': None}, {'have': None}, {'go': None}, {'there': None}],
 [{'i': None}, {'will': None}, {'be': None}, {'do': None}, {'that': None}],
 [{'i': None}, {'make': None}, {'it': None}, {'up': None}]]

In [338]:
adjusted_test_text = ['i have got married', 'i have gone there', 'i will be doing that', 'i made it up', 'doing that is not my hobby','I was doing it whole day', 'I will pick you up']
grammar_map_test = []
for sentence in adjusted_test_text:
    sent_grammar_map = get_sent_gramm_features_map(sentence)
    grammar_map_test.append(sent_grammar_map)
grammar_map_test

[[('i', {74: 94, 'PronType': 'prs'}),
  ('have', {74: 99, 'Tense': 'pres', 'VerbForm': 'fin'}),
  ('get', {74: 99, 'Aspect': 'perf', 'Tense': 'past', 'VerbForm': 'part'}),
  ('marry', {74: 99, 'Aspect': 'perf', 'Tense': 'past', 'VerbForm': 'part'})],
 [('i', {74: 94, 'PronType': 'prs'}),
  ('have', {74: 99, 'Tense': 'pres', 'VerbForm': 'fin'}),
  ('go', {74: 99, 'Aspect': 'perf', 'Tense': 'past', 'VerbForm': 'part'}),
  ('there', {74: 85, 'Degree': 'pos'})],
 [('i', {74: 94, 'PronType': 'prs'}),
  ('will', {74: 99, 'VerbType': 'mod'}),
  ('be', {74: 99, 'VerbForm': 'inf'}),
  ('do', {74: 99, 'Aspect': 'prog', 'Tense': 'pres', 'VerbForm': 'part'}),
  ('that', {74: 89})],
 [('i', {74: 94, 'PronType': 'prs'}),
  ('make', {74: 99, 'Tense': 'past', 'VerbForm': 'fin'}),
  ('it', {74: 94, 'PronType': 'prs'}),
  ('up', {74: 93})],
 [('do', {74: 99, 'Aspect': 'prog', 'Tense': 'pres', 'VerbForm': 'part'}),
  ('that', {74: 89}),
  ('be',
   {74: 99,
    'Number': 'sing',
    'Person': 3,
    'Ten

In [351]:
def get_difficult_grammar(text_grammar_map, weights_list, debug = False):
    for sentence_grammar_map, sentence_weights in zip(text_grammar_map,weights_list):
        for el_ind in range(len(sentence_grammar_map)):
            #print(sentence_grammar_map[el_ind])
            
            if('Aspect' in sentence_grammar_map[el_ind][1]):
                #present perfect
                if ( sentence_grammar_map[el_ind][1]['Aspect'] == 'perf' ):
                    if(sentence_grammar_map[el_ind - 2][0] == 'have' or sentence_grammar_map[el_ind - 1][0] == 'have'):
                        if(debug): print("PRESENT PERFECT")
                        if(debug): print(sentence_grammar_map[el_ind - 1])
                        if(debug): print(sentence_grammar_map[el_ind])

                        sentence_weights[el_ind]["diff_grammar"] = "pr_perf"
                        sentence_weights[el_ind - 1]["diff_grammar"] = "pr_perf"
                
                
                elif(sentence_grammar_map[el_ind][1]['Aspect'] == 'prog'):
                    #future continious check "will + be + v-ing"
                    if (sentence_grammar_map[el_ind - 1][0] == 'be' and sentence_grammar_map[el_ind - 2][0] == 'will'):
                        if(debug): print("FUTURE CONTINIOUS")                       
                        if(debug): print("prev word is", sentence_grammar_map[el_ind - 2])
                        if(debug): print("prev word is", sentence_grammar_map[el_ind - 1])
                        if(debug): print(sentence_grammar_map[el_ind])
                        sentence_weights[el_ind]["diff_grammar"] = "fut_cont"
                        sentence_weights[el_ind - 1]["diff_grammar"] = "fut_cont"
                        sentence_weights[el_ind - 2]["diff_grammar"] = "fut_cont"
                   #past continious was/were + v-ing"     
                    elif (sentence_grammar_map[el_ind - 1][0] == 'be' and sentence_grammar_map[el_ind -1][1]['Tense'] == 'past'):
                        if(debug): print("PAST CONTINIOUS")
                        if(debug): print(sentence_grammar_map[el_ind])
                        if(debug): print("prev word is", sentence_grammar_map[el_ind - 1])
                        sentence_weights[el_ind]["diff_grammar"] = "past_cont"
                        sentence_weights[el_ind - 1]["diff_grammar"] = "past_cont"
                        
                
    return weights_list

get_difficult_grammar(grammar_map_test,test_weights, debug = True)

PRESENT PERFECT
('have', {74: 99, 'VerbForm': 'fin', 'Tense': 'pres'})
('get', {74: 99, 'VerbForm': 'part', 'Tense': 'past', 'Aspect': 'perf'})
PRESENT PERFECT
('get', {74: 99, 'VerbForm': 'part', 'Tense': 'past', 'Aspect': 'perf'})
('marry', {74: 99, 'VerbForm': 'part', 'Tense': 'past', 'Aspect': 'perf'})
PRESENT PERFECT
('have', {74: 99, 'VerbForm': 'fin', 'Tense': 'pres'})
('go', {74: 99, 'VerbForm': 'part', 'Tense': 'past', 'Aspect': 'perf'})
FUTURE CONTINIOUS
prev word is ('will', {74: 99, 'VerbType': 'mod'})
prev word is ('be', {74: 99, 'VerbForm': 'inf'})
('do', {74: 99, 'VerbForm': 'part', 'Tense': 'pres', 'Aspect': 'prog'})
PAST CONTINIOUS
('do', {74: 99, 'VerbForm': 'part', 'Tense': 'pres', 'Aspect': 'prog'})
prev word is ('be', {74: 99, 'VerbForm': 'fin', 'Tense': 'past'})


[[{'weight': 0, 'word': 'i'},
  {'diff_grammar': 'pr_perf', 'weight': 0, 'word': 'have'},
  {'diff_grammar': 'pr_perf', 'weight': 0, 'word': 'get'},
  {'diff_grammar': 'pr_perf', 'weight': 0, 'word': 'marry'}],
 [{'weight': 0, 'word': 'i'},
  {'diff_grammar': 'pr_perf', 'weight': 0, 'word': 'have'},
  {'diff_grammar': 'pr_perf', 'weight': 0, 'word': 'go'},
  {'weight': 0, 'word': 'there'}],
 [{'weight': 0, 'word': 'i'},
  {'diff_grammar': 'fut_cont', 'weight': 0, 'word': 'will'},
  {'diff_grammar': 'fut_cont', 'weight': 0, 'word': 'be'},
  {'diff_grammar': 'fut_cont', 'weight': 0, 'word': 'do'},
  {'weight': 0, 'word': 'that'}],
 [{'weight': 0, 'word': 'i'},
  {'weight': 0, 'word': 'make'},
  {'weight': 0, 'word': 'it'},
  {'weight': 0, 'word': 'up'}],
 [{'weight': 0, 'word': 'do'},
  {'weight': 0, 'word': 'that'},
  {'weight': 0, 'word': 'be'},
  {'weight': 0, 'word': 'not'},
  {'weight': 0, 'word': 'my'},
  {'weight': 0, 'word': 'hobby'}],
 [{'weight': 0, 'word': 'I'},
  {'diff_gramm

In [359]:
def get_phrasal_verbs(text_grammar_map, weights_list, debug = False):
    for sentence_grammar_map, sentence_weights in zip(text_grammar_map,weights_list):
        for el_ind in range(1, len(sentence_grammar_map)):  
            for searh_ind in range(1, 3):
                try_phrase = sentence_grammar_map[el_ind - searh_ind][0] + ' ' + sentence_grammar_map[el_ind][0]
                #print(try_phrase)
                if(try_phrase in phrasal_list):
                    if(debug): print("Phrasal Verb found:", try_phrase)
                    sentence_weights[el_ind]["phrasal_verb"] = "yes"
                    sentence_weights[el_ind - searh_ind]["phrasal_verb"] = "yes"
    return weights_list
        #print("\n")
get_phrasal_verbs(grammar_map_test,test_weights,debug = True)

Phrasal Verb found: make up
Phrasal Verb found: pick up


[[{'weight': 0, 'word': 'i'},
  {'diff_grammar': 'pr_perf', 'weight': 0, 'word': 'have'},
  {'diff_grammar': 'pr_perf', 'weight': 0, 'word': 'get'},
  {'diff_grammar': 'pr_perf', 'weight': 0, 'word': 'marry'}],
 [{'weight': 0, 'word': 'i'},
  {'diff_grammar': 'pr_perf', 'weight': 0, 'word': 'have'},
  {'diff_grammar': 'pr_perf', 'weight': 0, 'word': 'go'},
  {'weight': 0, 'word': 'there'}],
 [{'weight': 0, 'word': 'i'},
  {'diff_grammar': 'fut_cont', 'weight': 0, 'word': 'will'},
  {'diff_grammar': 'fut_cont', 'weight': 0, 'word': 'be'},
  {'diff_grammar': 'fut_cont', 'weight': 0, 'word': 'do'},
  {'weight': 0, 'word': 'that'}],
 [{'weight': 0, 'word': 'i'},
  {'phrasal_verb': 'yes', 'weight': 0, 'word': 'make'},
  {'phrasal_verb': 'yes', 'weight': 0, 'word': 'it'},
  {'phrasal_verb': 'yes', 'weight': 0, 'word': 'up'}],
 [{'weight': 0, 'word': 'do'},
  {'weight': 0, 'word': 'that'},
  {'weight': 0, 'word': 'be'},
  {'weight': 0, 'word': 'not'},
  {'weight': 0, 'word': 'my'},
  {'weight

In [238]:
doc = nlp("I were going there")
for token in doc:
    if(token.lemma_[0] == "-"):
        token.lemma_ = token.text
    print(token.text, token.lemma_, nlp.vocab.morphology.tag_map[token.tag_])

I I {74: 94, 'PronType': 'prs'}
were be {74: 99, 'VerbForm': 'fin', 'Tense': 'past'}
going go {74: 99, 'VerbForm': 'part', 'Tense': 'pres', 'Aspect': 'prog'}
there there {74: 85, 'Degree': 'pos'}


# СЧИТАЕМ 

In [ ]:
сначала даем веса фразовым глаголам
затем даем веса по тфидф если веса нет то даем 0,1
затем 

In [ ]:
ИТЕРРИРУЕМСЯ ПО ЛЕММАТИЗИРОВАННОМУ ТЕКСТУ ВНУТРИ КАЖДОГО ПРЕДЛОЖЕНИЯ
ПРОВЕРЯЕМ ГРАММ ФИЧИ ПО СООТВЕТСВУЮЩЕУ ИНДЕКСУ
ТАКЖЕ ПРОВЕРЯЕМ ТФДФ ИНДЕКСЫ
ЕСЛИ СЛОВО ОТСУТСТВУЕТ В ТФИДФ ТО ДАЕМ ЕМУ МИНИМАЛЬНЫЙ ВЕС ТИПА 0,1

In [190]:
grammar_map[0][0][1]

{74: 89}

In [325]:
grammar_map

[[('this', {74: 89}),
  ('be',
   {74: 99,
    'Number': 'sing',
    'Person': 3,
    'Tense': 'pres',
    'VerbForm': 'fin'}),
  ('my', {74: 83, 'Poss': 'yes', 'PronType': 'prs'}),
  ('friend', {74: 91, 'Number': 'sing'}),
  ('jimmy', {74: 91, 'Number': 'sing'})],
 [('he', {74: 94, 'PronType': 'prs'}),
  ('be',
   {74: 99,
    'Number': 'sing',
    'Person': 3,
    'Tense': 'pres',
    'VerbForm': 'fin'}),
  ('from', {74: 84}),
  ('india', {74: 91, 'Number': 'sing'})],
 [('jimmy', {74: 91, 'Number': 'sing'}),
  ('be',
   {74: 99,
    'Number': 'sing',
    'Person': 3,
    'Tense': 'pres',
    'VerbForm': 'fin'}),
  ('a', {74: 89}),
  ('vegetarian', {74: 91, 'Number': 'sing'})],
 [('the', {74: 89}),
  ('good', {74: 83, 'Degree': 'sup'}),
  ('breakfast', {74: 91, 'Number': 'sing'}),
  ('for', {74: 84}),
  ('him', {74: 94, 'PronType': 'prs'}),
  ('be',
   {74: 99,
    'Number': 'sing',
    'Person': 3,
    'Tense': 'pres',
    'VerbForm': 'fin'}),
  ('a', {74: 89}),
  ('glass', {74: 91, 

In [361]:
tf_idf_dict

{'accompany': 0.323,
 'add': 0.39,
 'ago': 0.488,
 'agree': 0.371,
 'also': 0.323,
 'apple': 0.341,
 'ask': 0.514,
 'banana': 0.341,
 'beautiful': 0.415,
 'boil': 0.447,
 'breakfast': 0.341,
 'bridesmaid': 0.514,
 'brush': 0.298,
 'bunch': 0.323,
 'buy': 0.415,
 'cake': 0.189,
 'calorie': 0.39,
 'carry': 0.323,
 'ceremony': 0.371,
 'church': 0.207,
 'clothe': 0.232,
 'confetti': 0.232,
 'cook': 0.447,
 'could': 0.5,
 'day': 0.298,
 'diet': 0.844,
 'dieting': 0.707,
 'dream': 0.441,
 'dress': 0.266,
 'easy': 0.707,
 'eat': 0.176,
 'even': 0.573,
 'every': 0.408,
 'everyone': 0.457,
 'everything': 0.39,
 'fish': 0.447,
 'fit': 0.5,
 'flower': 0.323,
 'friend': 0.774,
 'fruit': 0.512,
 'fry': 0.505,
 'get': 0.529,
 'give': 0.441,
 'glass': 0.341,
 'go': 0.408,
 'good': 0.341,
 'great': 0.457,
 'green': 0.378,
 'guest': 0.232,
 'gym': 0.408,
 'hair': 0.298,
 'hard': 0.598,
 'help': 0.298,
 'home': 0.5,
 'husband': 0.232,
 'india': 1.0,
 'inside': 0.426,
 'invitation': 0.774,
 'jimmy': 0.43

In [362]:
def assign_tf_idf(text_grammar_map, weights_list, debug = False):
    for sentence_grammar_map, sentence_weights in zip(text_grammar_map,weights_list):
        for el_ind in range(len(sentence_grammar_map)):
            lemma = sentence_grammar_map[el_ind][0]
            if(lemma in tf_idf_dict):
                sentence_weights[el_ind]["weight"] = tf_idf_dict[lemma]
            else:
                sentence_weights[el_ind]["weight"] = 0.1
    return weights_list

In [388]:
ex_text_weights = get_weights_empty_list(grammar_map)
ex_text_weights = get_difficult_grammar(grammar_map, ex_text_weights, debug = True)
ex_text_weights = get_phrasal_verbs(grammar_map, ex_text_weights, debug = True)
ex_text_weights = assign_tf_idf(grammar_map, ex_text_weights, debug = True)
ex_text_weights

Phrasal Verb found: agree with
Phrasal Verb found: add up
Phrasal Verb found: give up


[[{'weight': 0.1, 'word': 'this'},
  {'weight': 0.1, 'word': 'be'},
  {'weight': 0.1, 'word': 'my'},
  {'weight': 0.774, 'word': 'friend'},
  {'weight': 0.439, 'word': 'jimmy'}],
 [{'weight': 0.1, 'word': 'he'},
  {'weight': 0.1, 'word': 'be'},
  {'weight': 0.1, 'word': 'from'},
  {'weight': 1.0, 'word': 'india'}],
 [{'weight': 0.439, 'word': 'jimmy'},
  {'weight': 0.1, 'word': 'be'},
  {'weight': 0.1, 'word': 'a'},
  {'weight': 0.774, 'word': 'vegetarian'}],
 [{'weight': 0.1, 'word': 'the'},
  {'weight': 0.341, 'word': 'good'},
  {'weight': 0.341, 'word': 'breakfast'},
  {'weight': 0.1, 'word': 'for'},
  {'weight': 0.1, 'word': 'him'},
  {'weight': 0.1, 'word': 'be'},
  {'weight': 0.1, 'word': 'a'},
  {'weight': 0.341, 'word': 'glass'},
  {'weight': 0.1, 'word': 'of'},
  {'weight': 0.341, 'word': 'orange'},
  {'weight': 0.341, 'word': 'juice'},
  {'weight': 0.277, 'word': 'two'},
  {'weight': 0.341, 'word': 'apple'},
  {'weight': 0.1, 'word': 'and'},
  {'weight': 0.277, 'word': 'three

In [384]:
def split_into_groups(text_weights):
    difficult_vocabulary = []
    easy_vocabulary = []
    for sentence_weights in text_weights:
        for word_weight in sentence_weights:
            #print(word_weight)
            if('diff_grammar' in word_weight or 'phrasal_verb' in word_weight or word_weight['word'] not in basic_vocabulary):
                difficult_vocabulary.append(word_weight)
            
            else:
                easy_vocabulary.append(word_weight)
    return easy_vocabulary, difficult_vocabulary
                
easy, difficult = split_into_groups(ex_text_weights)

In [385]:
len(easy)

252

In [386]:
difficult

[{'weight': 0.774, 'word': 'vegetarian'},
 {'weight': 0.426, 'word': 'watermelon'},
 {'weight': 0.426, 'word': 'watermelon'},
 {'weight': 0.598, 'word': 'soft'},
 {'weight': 0.774, 'word': 'juicy'},
 {'weight': 0.746, 'word': 'lovely'},
 {'weight': 0.488, 'word': 'receive'},
 {'weight': 0.774, 'word': 'invitation'},
 {'weight': 0.361, 'word': 'wedding'},
 {'weight': 0.529, 'word': 'marry'},
 {'weight': 0.1, 'word': 'such'},
 {'weight': 0.371, 'word': 'ceremony'},
 {'weight': 0.232, 'word': 'guest'},
 {'weight': 0.232, 'word': 'clothe'},
 {'weight': 0.207, 'word': 'church'},
 {'weight': 0.232, 'word': 'throw'},
 {'weight': 0.232, 'word': 'confetti'},
 {'weight': 0.232, 'word': 'tasty'},
 {'weight': 0.232, 'word': 'video'},
 {'weight': 0.514, 'word': 'bridesmaid'},
 {'weight': 0.298, 'word': 'mean'},
 {'weight': 0.361, 'word': 'wedding'},
 {'weight': 0.266, 'word': 'must'},
 {'weight': 0.361, 'word': 'wedding'},
 {'weight': 0.298, 'word': 'brush'},
 {'weight': 0.266, 'word': 'must'},
 {'